In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler, select_urls
from IPython.display import display
from IPython.core.debugger import set_trace

In [2]:
crawler = NewsCrawler()#'vice')

# 1. URL을 모은다

In [3]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [29]:
collect_results = crawler.collect(); display(*collect_results)

100.00% (60.96 seconds): Collecting URLs... fox                 

n_total     19725
n_unique    19657
dtype: int64

,pubs,actual_pub
https://businessinsider.co.za/apple-is-considering-music-and-tv-plus-subscription-bundle-report-2019-10,"businessinsider, news24",businessinsider
https://businessinsider.co.za/business-insider-daily-update-2019-9,"businessinsider, news24",businessinsider
https://cnbc.com/2019/10/09/trump-rips-nba-players-officials-for-pandering-to-china-but-says-league-has-to-work-out-dispute-out-on-its-own.html,"cnbc, axios",cnbc
https://hotair.com/archives/ed-morrissey/2019/10/09/media-honey-trap-doj-indicts-intel-expert-leaking-journalist-lover,"townhall, hotair",hotair
https://msnbc.com/morning-joe/watch/congressman-on-why-he-s-threatening-pompeo-s-salary-70923845715,"msnbc, nbcnews",msnbc
https://msnbc.com/morning-joe/watch/joe-obstructing-an-inquiry-likely-to-further-damage-trump-s-standing-70912069958,"msnbc, nbcnews",msnbc
https://msnbc.com/morning-joe/watch/trump-being-used-by-giuliani-says-conservative-writer-70913093972,"msnbc, nbcnews",msnbc
https://msnbc.com/morning-joe/watch/what-is-the-legal-remedy-for-trump-s-refusal-to-cooperate-70908997891,"msnbc, nbcnews",msnbc
https://msnbc.com/msnbc/watch/watch-courtney-kube-s-son-wanders-on-set-during-live-reporting-70918725778,"msnbc, nbcnews",msnbc
https://msnbc.com/the-beat-with-ari/watch/-crazy-ukraine-memo-shows-wh-official-shaken-by-ukraine-call-70953541708,"msnbc, nbcnews",msnbc


huffpost                  71
cnn                      815
investing.com             35
politico                 216
time                      17
cnbc                     162
fox                      339
bbc                      271
businessinsider          541
morningstar               84
wsj                      102
nytimes                  105
guardian                 102
reuters                 1151
washingtontimes          471
washingtonpost           124
cbs                      399
marketwatch              156
atlantic                 102
vice                      44
npr                       27
newrepublic               20
yahoo                    139
independent              158
heritage                 286
zdnet                    385
townhall                 627
abcnews                  102
hotair                    84
cbc                      296
                        ... 
metro                    233
msnbc                    103
nationalreview            44
news24        

In [25]:
collect_results[2]['thinkprogress']

82

# 2. 모은 URL중 이미 받은 것은 버린다

In [30]:
select_results = crawler.select(); display(*select_results)

100.00% (2.28 seconds): Selecting URLs... cryptonews          

n_total     87
n_unique    87
dtype: int64

,pubs,actual_pub


huffpost                 0
cnn                      0
investing.com            1
politico                 0
time                     0
cnbc                     1
fox                      0
bbc                      1
businessinsider          7
morningstar              0
wsj                      0
nytimes                  1
guardian                 3
reuters                 15
washingtontimes          1
washingtonpost           0
cbs                      0
marketwatch              1
atlantic                 0
vice                     0
npr                      0
newrepublic              0
yahoo                   25
independent              1
heritage                 0
zdnet                    0
townhall                 1
abcnews                  0
hotair                   0
cbc                      0
                        ..
metro                    0
msnbc                    0
nationalreview           0
news24                   1
techcrunch               0
arstechnica              0
e

In [27]:
crawler.urls_selected['thinkprogress']

set()

# 3. 해당 URL의 뉴스를 다운로드한다

In [31]:
download_results = crawler.download(); download_results

100.00% (66.66 seconds): Downloading... yahoo               

,downloaded,trashed
abcnews,0,0
afr,0,0
aljazeera,0,1
americanconservative,0,0
arirang,1,0
arstechnica,0,0
arynews,0,0
atlantic,0,0
axios,0,0
bbc,1,0


In [20]:
import hashlib
[hashlib.sha1(url.encode('utf-8')).hexdigest() for url in crawler.urls_final['japan-news']['trashed']]

['d80494a4761ead4276e510dc4f64762e319902e0',
 'f25b1d06e58c05786ad57fe9bea9817859de3861',
 '02d668060004e25c2299c9a23445e38a9c6ec771',
 '2d3e330325b35d2c622806feac055478a4a61559',
 '2e87c621d1ffb7f28485496eb82814891ff5dd1b',
 '40cf0a5a2b87e040cd211da1855cb615702cb0ca',
 'feefa98b143ce76ef3cadf0a2764ca634f773fee',
 '0a900188f145b48aad8c70bde741aea563dba542',
 'ef7e33fa30c8be4b6d90681b383234ee4f73e51a',
 '64cc9a5e5a51520306154fda70d482b57a7420b0',
 '12e43a97340dc5a9f9270b38a9705e461456041b',
 'd71b4a23216b50deb276d3ccd8c1905939f5a9c2',
 '9a5e5f813637acd5359da986e562fa9ad9858f4b',
 '44c36bff1fe4d64fb468131f1221959fa30718d6',
 'dbf61f3dd462f9c650d0b2f42ca69717701fc7d1',
 '385dbc7ad54410ac2078055044e34c6e30cfde4b',
 'a31fe291585e194540805022e45df9ebdbabc9fc',
 '66afae226b2dc5d329fc731f7931f9f6d6f37e23']